<a href="https://colab.research.google.com/github/etgins/Mice_ASD_Detection/blob/main/audio_feature_extraction_REDUCTION_BY_RECORDING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----------------------------------------------
Written by Itamar Ginsberg & Alon Schreuer, October 2021



# **1. Import data and pre-process**

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

## FOR COLAB USE:
## ITAMAR:

# old, incomplete dataset
# dataset = pd.read_excel('/content/drive/MyDrive/Project_A/Project_A_files/total_data UPDATED EM 020821.xlsx')

# new dataset:
dataset = pd.read_excel('/content/drive/MyDrive/Project_A/Project_A_files/complete_data_131221.xlsx')

# in case we need to convert from xls to csv:
#dataset.to_csv (r'/content/drive/MyDrive/Project_A/Project_A_files/total_data UPDATED EM 020821.csv', index = None, header=True)

# extract only the relevant columns / features
X = dataset[["Name", "Start Point (Hz)", "End Point (Hz)", "Duration (time)", "Syllable number", "Recording Number", "Mother Genotype", "Sex", "ISI_time", "Offspring Genotype"]]



#missing data
print("--- CALCULATING AMOUNT OF MISSING DATA ---")
total = X.isnull().sum().sort_values(ascending=False)
percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data.head(20))



# drop data with missing values from dataset or X - TODO: find a better solution / retrieve from research team
X = X.dropna(axis = 0, how='any')
# print(X)


# encode binary feature - mother genotype
y = X[["Mother Genotype"]]
# print(y)
# encode labels
l1 = LabelEncoder()
l1.fit(y)
y = l1.transform(y)
X[["Mother Genotype"]] = y
# print(X)

# encode binary feature - offspring genotype
y = X[["Offspring Genotype"]]
# print(y)
# encode labels
l1 = LabelEncoder()
l1.fit(y)
y = l1.transform(y)
X[["Offspring Genotype"]] = y
# print(X)

# encode binary feature - sex
y = X[["Sex"]]
# print(y)
# encode labels
l1 = LabelEncoder()
l1.fit(y)
y = l1.transform(y)
X[["Sex"]] = y
# print(X)


# convert duration to micro-seconds from 'datetime.time' type to 'timedelta' object
# from datetime import datetime
# duration_vec = X[["Duration (time)"]]
# # print(duration_vec)
# duration_vec2 = np.array(duration_vec)
# # print(duration_vec2)
# duration_vec3 = np.zeros([1, np.size(duration_vec2)])
# # print(duration_vec3)
# for i in range (len(duration_vec)):
#   duration_single = duration_vec2[i,0]
#   # print(duration_single)
#   # duration_vec3[0,i] = int(duration_single.strftime(format = '%f'))
#   # print(duration_vec3[0,i])
# # X[["Duration (s)"]] = duration_vec2
# X[["Duration (s)"]] = np.transpose(duration_vec2)
# print(X)




X.describe()



Mounted at /content/drive
--- CALCULATING AMOUNT OF MISSING DATA ---
                    Total   Percent
End Point (Hz)       1332  0.186921
Start Point (Hz)     1332  0.186921
ISI_time             1266  0.177659
Recording Number      139  0.019506
Syllable number       139  0.019506
Duration (time)       139  0.019506
Sex                     4  0.000561
Offspring Genotype      3  0.000421
Mother Genotype         3  0.000421
Name                    3  0.000421


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be 

,Start Point (Hz),End Point (Hz),Duration (time),Syllable number,Mother Genotype,Sex,ISI_time,Offspring Genotype
count,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000
mean,69358.769020,66319.918736,0.065567,6.614549,0.219877,0.370697,0.143322,0.710041
std,14998.931689,16641.998968,0.021516,2.647647,0.414206,0.483041,0.095122,0.453790
min,35971.122900,26294.312700,0.000010,1.000000,0.000000,0.000000,0.000900,0.000000
25%,56889.327000,52890.550000,0.054167,3.000000,0.000000,0.000000,0.100790,0.000000
50%,71026.754200,67264.212400,0.067015,8.000000,0.000000,0.000000,0.119600,1.000000
75%,79452.927000,77735.988125,0.079650,9.000000,0.000000,1.000000,0.145300,1.000000
max,180976.587000,180558.526800,0.205710,10.000000,1.000000,1.000000,1.793500,1.000000


# **2. Setup for feature extraction**

---

In [ ]:
def take_data_by_name(name_searched, Data, name_list):
  # print(name_list)
  ind = name_list == name_searched
  # print(ind)
  matching_data = Data[ind].reset_index()
  
  # print(matching_data)
  return matching_data


def take_data_by_recording(recording_searched, Data, recording_list):
  # print('recording list: ', recording_list, '\n')
  ind = recording_list == recording_searched
  # print('matching indices: ', ind, '\n')
  matching_data = Data[ind].reset_index()
  
  # print('matching data: ', matching_data, '\n')
  return matching_data



# find set of mouse names
name_column = X["Name"]
unique_name_set = set(name_column)
unique_name_list = list(unique_name_set)
# unique_name_list = unique_name_list[0:-1]   # use if first entry is column name

number_of_mice = len(unique_name_list)
# print(unique_name_list)
# print(number_of_mice, 'mice in data')

# if needed - remove nan from list of names and update
unique_name_list = [x for x in unique_name_list if str(x) != 'nan']
number_of_mice = len(unique_name_list)

print(unique_name_list)
print(number_of_mice, 'mice in data')


# SAVED FOR FUTURE REFERENCE - NOT USED (instead, each mouse is split into recordings below)
# # find set of recordings
# recording_column = X["Recording Number"]
# unique_recording_set = set(recording_column)
# unique_recording_list = list(unique_recording_set)
# # unique_name_list = unique_name_list[0:-1]   # first entry is column name

# number_of_recordings = len(unique_recording_list)
# # print(unique_name_list)
# # print(number_of_mice, 'mice in data')

# # remove nan from list of recordings
# unique_recording_list = [x for x in unique_recording_list if str(x) != 'nan']
# number_of_recordings = len(unique_recording_list)

# print(unique_recording_list)
# print(number_of_recordings, 'recordings in data')

['08172', '08136', '17453I', '08134', '08139', '08166', 8096, '08141', 8098, 17444, 8101, 8102, 8103, 8104, 8105, 8106, 8107, 17450, 17453, 17454, 17455, 17456, 8111, '08056', '08103', '08119', '08106', '08169', 8119, 8121, 17470, 17471, 17472, 17474, 8130, 8132, 17477, 8134, 17479, 8136, 8137, 8139, 17484, '08121', 17489, 17490, 17491, '08096', '08111', '08132', '08059', 8027, 8028, '08105', '08098', 8169, '08104', '08102', '08057', '08107', 8056, 8057, '08060', 8059, 8060, '08101']
66 mice in data


Dataset statistics (basic)
----

In [ ]:
# HOW MANY SAMPLES FOR EACH GENOTYPE
print('HOW MANY SAMPLES FOR EACH GENOTYPE:')
groupby_count1 = X.groupby(['Offspring Genotype']).count()
print(groupby_count1.iloc[:,0], '\n')

# HOW MANY SAMPLES FOR EACH SYLLABLE
print('HOW MANY SAMPLES FOR EACH SYLLABLE:')
groupby_count2 = X.groupby(['Syllable number']).count()
print(groupby_count2.iloc[:,0], '\n')

# EACH GENOTYPE MEAN ISI TIME
print('EACH GENOTYPE MEAN ISI TIME:')
a = X.groupby('Offspring Genotype', as_index=False)["ISI_time"].mean()
print(a, '\n')

HOW MANY SAMPLES FOR EACH GENOTYPE:
Offspring Genotype
0    1415
1    3465
Name: Name, dtype: int64 

HOW MANY SAMPLES FOR EACH SYLLABLE:
Syllable number
1.0      260
2.0      279
3.0      703
4.0       34
5.0       15
6.0       34
7.0      841
8.0     1484
9.0     1122
10.0     108
Name: Name, dtype: int64 

EACH GENOTYPE MEAN ISI TIME:
   Offspring Genotype  ISI_time
0                   0  0.137236
1                   1  0.145807 



# **3. Extract features**

---

In [ ]:
  # final data is made up of: 
    # 0-9    avg. start freq. per syllable
    # 10-19  avg. end freq. per syllable
    # 20-29  syllable distribution - percentage of each syllable
    # 30-39  avg. syllable duration per syllable
    # 40     mother genotype
    # 41     pup sex
    # 42     mean ISI time (time between syllables)
    # ? Bandwidth - TBD
    
    # 43     label: offspring genotype

num_of_syllables = 10
final_data_size = 4*num_of_syllables + 1 + 1 + 1 

# Initialize final data variable: mouse_final_data
# Start with one row and add one each iteration
mouse_final_data = np.zeros([1,(final_data_size+1)])

""" 
Work on each mouse and each of its recordings separately and calculate all features.
Place the results in mouse_final_data[row,:]
"""

row = 0   # output data row - each mouse has a row for each of its recordings

for idx in range(0,number_of_mice):
    mouse_split_data = take_data_by_name(unique_name_list[idx], X, name_column)
    print('mouse name:', unique_name_list[idx],', ' 'idx:', idx)
    # print('matching data:', '\n',mouse_split_data)
    # print('headers are: ', list(mouse_split_data.columns))

    # find set of recordings this specific mouse has
    ## -------------------------------------------
    recording_column = mouse_split_data["Recording Number"]
    unique_recording_set = set(recording_column)
    unique_recording_list = list(unique_recording_set)
    # unique_recording_list = unique_recording_list[0:-1]   # use if first entry is column name

    # remove nan from list of recordings
    unique_recording_list = [x for x in unique_recording_list if str(x) != 'nan']
    number_of_recordings = len(unique_recording_list)
    
    print('these are the unique recordings for this mouse: ', unique_recording_list, '\n')
    print('total of: ', number_of_recordings, 'recordings \n')




    for recording in range (number_of_recordings):
      # take data by recording from single mouse we already split
      ## -------------------------------------------
      print('recording number:', recording, '\n')
      recording_split_data = take_data_by_recording(unique_recording_list[recording], mouse_split_data, recording_column)
      print('this is its data:', '\n', recording_split_data)


      # calculate feature 1 - average start freq for each syllable
      ## -------------------------------------------
      print("\n", "--- CALCULATING FEATURE 1: recording's average start freq for each syllable ---", "\n")
      """
      - take start frequencies grouped by syllable number, and calculate the average
      - fill a 10-long vector with the final result (and add zeros to non-existing syllables)
      - place in mouse_final_data[0:10]
      """

      start_frequencies = np.array(recording_split_data.groupby(by = 'Syllable number', as_index=False)["Start Point (Hz)"].mean())
      # print('start_frequencies :','\n',start_frequencies,'\n')
      
      feature1_vec = np.zeros([1,10])
      for i in range(len(start_frequencies)):
        feature1_vec[0,int(start_frequencies[i,0])-1] = start_frequencies[i,1]
      
      # PRINT RESULT:
      # print('each syllable mean start freq:','\n', feature1_vec, '\n')

      mouse_final_data[row,0:10] = feature1_vec
      # print('mouse_final_data[row] so far:',mouse_final_data[row])




      # calculate feature 2 - average end freq for each syllable
      ## -------------------------------------------
      print("\n", "--- CALCULATING FEATURE 2: recording's average end freq for each syllable ---", "\n")
      

      """
      ITAMAR:
      - take end frequencies grouped by syllable number, and calculate the average
      - fill a 10-long vector with the final result (and add zeros to non-existing syllables)
      - place in mouse_final_data[0:10]
      """

      end_frequencies = np.array(recording_split_data.groupby(by = 'Syllable number', as_index=False)["End Point (Hz)"].mean())
      # print('end_frequencies :','\n',end_frequencies, '\n')
      
      feature2_vec = np.zeros([1,10])
      for i in range(len(end_frequencies)):
        feature2_vec[0,int(end_frequencies[i,0])-1] = end_frequencies[i,1]
      
      # PRINT RESULT:
      # print('each syllable mean end freq:','\n', feature2_vec, '\n')

      mouse_final_data[row,10:20] = feature2_vec

      # print('mouse_final_data[row] so far:',mouse_final_data[row])




      ## -------------------------------------------
      # calculate feature 3 - syllable distribution
      ## -------------------------------------------
      print("\n", "--- CALCULATING FEATURE 3: recording's syllable distribution ---", "\n")

    # take syll numbers
      syllable_num_row = np.transpose(np.array(recording_split_data["Syllable number"]))
      # print(syllable_num_row)

      # create a vector of all-zeros, and change values only for syllables that have been recorded
      distribution = np.zeros([1,10])
      # print(distribution)

    # increment syllable count, using distribution vector (as a histogram)
      for i in range(len(syllable_num_row)):
        # print("iteration", i)
        syll = int(syllable_num_row[i])
        # print("found syllable", syll)
        # print("old distribution:", distribution)
        distribution[0,syll-1] += 1 
        # print("new distribution:", distribution)

    # normalize histogram to distribution
      distribution = distribution / distribution.sum()
      # if all are zeros, dividing by zero will create nan's - turn those into zeros
      distribution[np.isnan(distribution)] = 0  

      # PRINT RESULT:
      # print("final distribution:", "\n", distribution)

    # transfer results to mouse_final_data
      mouse_final_data[row,20:30] = distribution
      # print('mouse_final_data[row] so far:',mouse_final_data[row])




      ## -------------------------------------------
      # calculate feature 4 - average syllable duration for each syllable
      ## -------------------------------------------
      print("\n", "--- CALCULATING FEATURE 4: recording's average syllable duration for each syllable ---", "\n")
      
      # take the mean of the duration column for each syllable
      recording_duration = recording_split_data[["Name", "Syllable number","Duration (time)"]]
      # print('syllable duration: ', recording_duration, '\n')

      a = recording_duration.groupby('Syllable number', as_index=False)["Duration (time)"].mean()
      a = np.array(a)
      
      # PRINT RESULT:
      # print("for this mouse and this recording, each syllable's mean duration is:","\n", a)

      # create a vector of all-zeros, and change values only for syllables that have been recorded
      feature4_vec = np.zeros([1,10])
      # print(np.shape(a))
      for i in range (np.size(a,0)):
        # print('iteration ', i)
        syllable = int(a[i,0])
        duration = a[i,1]
        # print('syllable ',syllable)
        # print('duration ',duration)

        feature4_vec[0,syllable-1] = duration
        # print(" means_vec:", feature4_vec)
      print('feature4_vec: ', feature4_vec)
      mouse_final_data[row,30:40] = feature4_vec
      # print('mouse_final_data[row] so far:',mouse_final_data[row])


      ## -------------------------------------------
      # calculate feature 5 - mother genotype
      ## -------------------------------------------
      print("\n", "--- CALCULATING FEATURE 5: mother genotype ---", "\n")
      # we already split by mouse and recording, so this should be the same for all recording_split_data. Take the first
      mouse_final_data[row,-4] = recording_split_data["Mother Genotype"].iloc[0]


      ## -------------------------------------------
      # calculate feature 6 - pup sex
      ## -------------------------------------------
      print("\n", "--- CALCULATING FEATURE 6: pup sex ---", "\n")
      # we already split by mouse and recording, so this should be the same for all recording_split_data. Take the first
      mouse_final_data[row,-3] = recording_split_data["Sex"].iloc[0]
      # PRINT RESULT:
      # print("for this mouse and this recording, the sex is:","\n", mouse_final_data[row,-3])


      ## -------------------------------------------
      # calculate feature 7 - mean ISI time
      ## -------------------------------------------
      print("\n", "--- CALCULATING FEATURE 7: mean ISI time ---", "\n")
      # we already split by mouse and recording, so this should be the same for all recording_split_data. Take the first
      mouse_final_data[row,-2] = recording_split_data["ISI_time"].mean()
      # PRINT RESULT:
      # print("for this mouse and this recording, the mean ISI time is:","\n", mouse_final_data[row,-2])


      ## -------------------------------------------
      # calculate feature ? - Bandwidth (???) - TBD
      ## -------------------------------------------
      # print("\n", "--- CALCULATING FEATURE 5: Bandwidth ---", "\n")



      ## -------------------------------------------
      # find mouse's genotype label
      ## -------------------------------------------
      # we already split by mouse and recording, so this should be the same for all recording_split_data. Take the first
      mouse_final_data[row,-1] = recording_split_data["Offspring Genotype"].iloc[0]



      print("\n", "--- FINISHED features, show final data: --- (mouse ", idx, ", recording ", recording, ')\n')
      print(mouse_final_data[row,:], "\n")

      # prepare for next iteration - add new row
      mouse_final_data = np.append(mouse_final_data, np.zeros([1,final_data_size+1]), axis=0)
      row += 1


# after all iterations, last line is added, but not needed - delete it
mouse_final_data = np.delete(mouse_final_data, -1, 0)

print("\n", "--- FINISHED EXTRACTION FOR ALL MICE --- " ,"\n")
# print("\n", "--- FINAL DATA FOR ALL MICE --- ", "\n")
print(mouse_final_data, "\n")

# export data to csv file for further use - COLAB
np.savetxt("processed_data_for_final_classification_REDUCTION_BY_RECORDING.csv", X=mouse_final_data, delimiter=",")
!cp "processed_data_for_final_classification_REDUCTION_BY_RECORDING.csv" /content/drive/MyDrive/Project_A/Project_A_files

Streaming output truncated to the last 5000 lines.


 --- CALCULATING FEATURE 6: pup sex --- 


 --- CALCULATING FEATURE 7: mean ISI time --- 


 --- FINISHED features, show final data: --- (mouse  60 , recording  7 )

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.26900387e+04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 6.17825654e+04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.84600000e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.31130000e-01 1.00000000e+00] 

recording number: 8 

this is its data: 
    level_0  index  Name  ...  Sex  ISI_time  Offspring Genotype
0